In [1]:
from datasets import load_dataset
from utils.utils import prepare_data, create_dataloaders
import json
from transformers import AutoModel
import numpy as np
from utils.multitask_dataloader import MultitaskDataLoader
from model.train_eval import train, evaluate

/Users/misha/Documents/Tinkoff lab/envTinkoffNew/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
datasets = {
    "cola" : load_dataset("glue", "cola"),
    "sst2" : load_dataset("glue", "sst2"),
    "rte" : load_dataset("glue", "rte")
}

### Как подготовить данные

Чтобы подготовить данные, нужно указать 2 параметра: <code>mode</code>, <code>augmentation</code>. Первый может принимать три значения:
```python
"singleHead", "multiHead", "singleTask"
```
Первый - для мультизадачного классификатора с одной головой. Второй - с тремя, для каждой задачи своя. Третий - для монозадачного классификатора. 

Параметр <code>augmentation</code> влияет на то, будет ли сделана аугментация данных. Также, можно указать, какую часть от исходного размера датасета добавить при аугментации (<code>augmentation_proportion</code>), и размер уменьшенного датасета (<code>subset_size</code>), который можно использовать, чтобы проверить, работают ли методы.

In [6]:
data = prepare_data(datasets, 
                    subset_size = 1.0, 
                    mode = "singleHead", 
                    augmentation = False, 
                    augmentation_proportion = 0.25)

### Как запускать эксперименты
Чтобы запустить эксперименты, нужно указать метод (<code>method</code>) и параметр <code>task_names</code>, который указывает на то, какие датасеты подаются в dataloader. Для мультизадачных экспериментов нужно подавать названия всех задач (<code>list(data.keys())</code>). Для однозадачных - название одной задачи в списке. Например 
```python
task_names = ["cola"] # ["sst2"],["rte"]
```
Метод может принимать следующие значения:
```python
"SingleHeadMultitask", "MultiHeadMultitask", "cola_Singletask", "sst2_Singletask", "rte_Singletask"
```

Также, можно менять название модели предобученного трансформер-енкодера (<code>model_name</code>).

Другие модели можно поискать на [Hugging Face](https://huggingface.co).

In [4]:
with open("./model/config.json", "r") as f:
    config = json.load(f)

In [ ]:
model_name = "bert-base-uncased" # это можно менять
method = "SingleHeadMultitask" # это можно менять
task_names = list(data.keys()) # это можно менять

In [ ]:
#несколько assert-ов на всякий случай
assert method in config, f"There is no such method. Valid methods: {list(config.keys())}"
valid_task_names = [list(data.keys())] + [[key] for key in data]
assert task_names in valid_task_names, f"task_names should be one of {valid_task_names}"
if config[method]["mode"] == "singletask":
    assert len(task_names) == 1, "You should enter one of singletask task names"
else:
    assert len(task_names) == 3, "You should enter one of multitask task names"

In [16]:
checkpoints_path = "./checkpoints"
results = {name : [] for name in task_names}
for i in range(3):
    model = AutoModel.from_pretrained(model_name)
    dataloaders = create_dataloaders(data, batch_sizes = config[method]["batch_sizes"], model_name = model_name)
    if config[method]["mode"] != "singletask":
        train_dataloader = MultitaskDataLoader(task_names, dataloaders["train"])
        valid_dataloader = MultitaskDataLoader(task_names, dataloaders["validation"])
    else:
        task_index = config[method]["task_index"]
        train_dataloader = MultitaskDataLoader(task_names, [dataloaders["train"][task_index]])
        valid_dataloader = MultitaskDataLoader(task_names, [dataloaders["validation"][task_index]])
    train(train_dataloader,
          valid_dataloader,
          model, 
          config[method],
          method_name = method,
          checkpoints_path = checkpoints_path,
         )
    
    for i, task_name in enumerate(task_names):
        valid_dataloader = MultitaskDataLoader([task_name], [dataloaders["validation"][i]])
        score = evaluate(
                valid_dataloader,
                model,
                method_name = method, 
                checkpoints_path = checkpoints_path, 
                config = config[method])
        results[task_name].append(score)

In [9]:
for name in task_names:
    print(f"Score for {method} for {name}: {np.mean(results[name])} +- {np.std(results[name])}")